In [23]:
import duckdb
import pandas as pd
# Connect to DuckDB. This does not create a physical database file, 
# DuckDB operates in-memory by default but can also persist to disk.
conn = duckdb.connect(database='duckdb_data.duckdb', read_only=False)

ny_file = 'ny.parquet'
sf_file = 'sf.parquet' 
# Use DuckDB to read the Parquet file. This creates a temporary view that you can query.
conn.execute(f"CREATE OR REPLACE VIEW my_parquet_view AS SELECT * FROM '{sf_file}'")

# Now you can query the data as if it was a table in a database
result = conn.execute("SELECT * FROM my_parquet_view").fetchall()

# Display the results
print(result[:3])


[({'oid': '65e4f174a8350f04b8ddde87'}, ' CA', '1024 Pacheco St,San Francisco, CA 94116', 4, 4, 2524, 2495000, '1024 Pacheco St,San Francisco', '94116'), ({'oid': '65e4f174a8350f04b8ddde88'}, ' CA', '1567 Union St,San Francisco, CA 94123', 6, 5, 4309, 7495000, '1567 Union St,San Francisco', '94123'), ({'oid': '65e4f174a8350f04b8ddde89'}, ' CA', '967 Natoma St,San Francisco, CA 94103', 1, 1, 1862, 1450000, '967 Natoma St,San Francisco', '94103')]


In [24]:
column_info = conn.execute("SELECT * FROM my_parquet_view LIMIT 0").description

# Extracting and printing column names
column_names = [col[0] for col in column_info]
print(column_names)

['_id', 'city', 'property-address', 'property-baths', 'property-beds', 'property-floorSpace', 'property-price', 'street', 'zip_code']


In [25]:
conn.execute("""
CREATE TABLE IF NOT EXISTS sf_house_list (
    _id VARCHAR PRIMARY KEY,
    city VARCHAR,
    property_address VARCHAR,
    property_baths INTEGER,
    property_beds INTEGER,
    property_floorSpace INTEGER,
    property_price INTEGER,
    street VARCHAR,
    zip_code VARCHAR,
    -- FOREIGN KEY (zip_code) REFERENCES zip_codes(zip_code)
);
CREATE TABLE IF NOT EXISTS ny_house_list (
    _id VARCHAR PRIMARY KEY,
    city VARCHAR,
    property_address VARCHAR,
    property_baths INTEGER,
    property_beds INTEGER,
    property_floorSpace INTEGER,
    property_price INTEGER,
    street VARCHAR,
    zip_code VARCHAR,
    -- FOREIGN KEY (zip_code) REFERENCES zip_codes(zip_code)
)
""")
conn.execute(f"COPY ny_house_list FROM '{ny_file}' (FORMAT 'parquet')")
conn.execute(f"COPY sf_house_list FROM '{sf_file}' (FORMAT 'parquet')")

In [26]:

# Query the ny_house_list table to see the data
query_result = conn.execute("SELECT * FROM ny_house_list").fetchall()

# Convert the results to a Pandas DataFrame
df = pd.DataFrame(query_result, columns=[desc[0] for desc in conn.description])
# Display the DataFrame
print(df)
# Close the connection when done
conn.close()

                                   _id city  \
0    {'oid': 65e4f174a8350f04b8dddde1}   NY   
1    {'oid': 65e4f174a8350f04b8dddde2}   NY   
2    {'oid': 65e4f174a8350f04b8dddde3}   NY   
3    {'oid': 65e4f174a8350f04b8dddde4}   NY   
4    {'oid': 65e4f174a8350f04b8dddde5}   NY   
..                                 ...  ...   
159  {'oid': 65e4f174a8350f04b8ddde80}   NY   
160  {'oid': 65e4f174a8350f04b8ddde81}   NJ   
161  {'oid': 65e4f174a8350f04b8ddde82}   NJ   
162  {'oid': 65e4f174a8350f04b8ddde83}   NY   
163  {'oid': 65e4f174a8350f04b8ddde84}   NY   

                                      property_address  property_baths  \
0                     1074 Union St,Brooklyn, NY 11225               2   
1            150-15 72 Road UNIT 5K,Flushing, NY 11367               1   
2        162 Continental Pl #A,Staten Island, NY 10303               2   
3                      3322 Mickle Ave,Bronx, NY 10469               6   
4                       160 Fordham St,Bronx, NY 10464           